# Accessing & Indexing Argo Data

In [1]:
#All the imports I utalized to allow any code to run across any of my notebooks
import xarray as xr
xr.set_options(display_style="html", display_expand_attrs=False);
from matplotlib import pyplot as plt
plt.style.use('default')
import cartopy
import cartopy.crs as ccrs
import argopy
import numpy as np
import os, shutil
import pandas as pd
from pathlib import Path
import seawater as sw

## Accessing Argo Data

### Option 1 - Using ArgoDataFetcher (good for graphs)

This method will allow you to visual and sort through Argo float data that fits a certain criteria. I did find that there are issues of inconsistency if you want to sort through all the Argo floats, active and inactive though. Most of the time it gives you all active floats, but excludes inactive, and sometimes you will get different errors that are fixed with running the cell individually, so for determining what Argo floats match your parameters this isn't the best method. But I would recommend using this strategy for graphs after determining matching Argo floats from ArgoIndexFetcher!

In [2]:
#Make sure to import the DataFetcher itself, and then define argo_loader; I normally add this to the beginning of my notebooks with imports
from argopy import DataFetcher as ArgoDataFetcher
argo_loader = ArgoDataFetcher(src='erddap', parallel=True) #Chose between 'argovis' and 'erddap' for your data sourcing!
argo_loader

<datafetcher.erddap> 'No access point initialised'
Available access points: float, profile, region
Performances: cache=False, parallel=True
User mode: standard
Dataset: phy

The other big thing to consider is your source of data. I mainly used src='erddap' for my data because it was in a more raw form of the data from Argo floats and didn't have as many automatic restrictions like q1 quality control, plus I have run into the least amount of errors with it. But src='argovis' could work just as well depending on your purposes, especially if errdap crashes. Another potential source, specifically for ArgoIndexFetcher (considering argovis does not work with IndexFetcher) is also 'gdac' which I included in this notebook only because errdap was recieving maintainence at the time of upload.

#### You can ask for Argo floats that are limited by the following conditions: 
- Latitude, Longitude, Depth
- Time, but would not recommend as there are issues with that variable
- Float WMO ID Number

In [ ]:
#Limiting Argo Floats by Latitude, Longitude, Depth
argo1 = argo_loader.region([-147, -138, 24, 36, 0, 10000]).load().data #(max longitude, min longitude, min latitude, max latitude, min depth, max depth)
argo1 #This is to visualize the information you found in a table format!

In [ ]:
#Limited Argo Floats by Float WMO ID Number
argo2 = argo_loader.float(5903385).load().data #You can also create a list of floats by replacing 5903385 with [5903385, 5903385]
argo2

In [ ]:
#Limited Argo Floats by Time in addition to Latitude, Longitude, and Depth
argo3 = argo_loader.region([-75, -45, 20, 30, 0, 10, '1999-01', '2023-08']).load().data #(max longitude, min longitude, min latitude, max latitude, min depth, max depth, max time, min time), time is in year, month, day.
argo3

#### This method of accessing Argo data has some built in functions and graphs you can utalize and build off too.

Different Graphs that I explored during this project will each get their own notebook, but here is the overall list for your convienence:

- Built-In Trajectory Graphs on a World Map
- Trajectory Graphs Mapped Against Topographical Map
- Trajectory Graphs color-coded by year launched for each profile!
- CTD graphs w/ Temperature and Salinity Mapped Against Depth
- T/S Diagrams w/ density color bars calculated from seawater Python package

#### Failed Attempt at Indexing

At first I tried to sort through Argo data that fit the following restrictions of EM-APEX's overall bounding box, and then list the floats satisfying those conditions using the world map trajectory graph like the following:

EM-APEX Conditions - Overall Bounding Box:
- lon max, lon min, lat min, lat max, depth min, depth max
- -146.321753, -139.39713, 25.633673, 35.469333, 0, 10000

In [ ]:
argo1 = argo_loader.region([-146.321753, -139.39713, 25.633673, 35.469333, 0, 10000]).load().data #(max longitude, min longitude, min latitude, max latitude, min depth, max depth)
#argo1.plot('trajectory', set_global=True) #This is to visualize the information you found in a table format!
argo1.plot('trajectory', set_global=True)

However, this process for the EM-APEX SMILE, NISKINE, and DIMES experiments was time consuming and unreliable, and so I used the IndexFetcher instead!

### Option 2 - Using ArgoIndexFetcher

This method will allow you to visual and sort through Argo float data that fits a certain criteria, but without the issues the DataIndexFetcher was running into. Since I was focusing on the EM-APEX SMILE Experiment, I chose to have four bounding boxes to investigate in and outside of the year 2017!

Bounding Boxes: (lat min, lat max, lon min, lon max)
- Overall = 25.633673, 35.469333, -146.321753, -139.39713
- Region #1 = 25.5, 28.0, -146.322, -145
- Region #2 = 28.0, 30.6, -146.322, -145
- Region #3 = 34.0, 35.469333, -140.5, -139.39713

#### First Import the IndexFetcher!

In [ ]:
from argopy import IndexFetcher as ArgoIndexFetcher

#### Then do the Indexing!

In [ ]:
#Overall Region - Argo floats that intersect EM-APEX SMILE experiment across all time
idx1 = ArgoIndexFetcher(src='erddap').region([-146.321753, -139.39713, 25.633673, 35.469333, '1999-01-01', '2023-01-01']).load()
idx1.index

In [ ]:
#This code tells you what floats agree with the limited parameters!
SMILE_floats_D1 = idx1.index.wmo #indexes into the column, wmo or float ID's
SMILE_floats_ND1 = SMILE_floats_D1.drop_duplicates() #gets rid of duplicates
SMILE_floats_ND1 #Tells you what these floats are

In [ ]:
len(SMILE_floats_ND1) #Tells you how many floats matched!

In [ ]:
#Overall Region - Argo floats that intersect EM-APEX SMILE experiment across 2017!
idx2 = ArgoIndexFetcher(src='erddap').region([-146.321753, -139.39713, 25.633673, 35.469333, '2017-01-01', '2017-12-31']).load()
idx2.index

In [ ]:
SMILE_floats_D2 = idx2.index.wmo #indexes into the column, wmo or float ID's
SMILE_floats_ND2 = SMILE_floats_D2.drop_duplicates() #gets rid of duplicates
SMILE_floats_ND2 #Tells you what these floats are
len(SMILE_floats_ND2) #Tells you how many floats matched!

In [ ]:
#Region 1 - Argo floats that intersect EM-APEX SMILE experiment across all time & limited time
idx3 = ArgoIndexFetcher(src='erddap').region([-146.322, -145, 25.5, 28.0, '1999-01-01', '2023-01-01']).load()
idx4 = ArgoIndexFetcher(src='erddap').region([-146.322, -145, 25.5, 28.0, '2017-01-01', '2017-12-31']).load() #MOST INTERESTING

In [ ]:
SMILE_floats_D3 = idx3.index.wmo #indexes into the column, wmo or float ID's
SMILE_floats_ND3 = SMILE_floats_D3.drop_duplicates() #gets rid of duplicates
SMILE_floats_ND3 #Tells you what these floats are
len(SMILE_floats_ND3)

In [ ]:
SMILE_floats_D4 = idx4.index.wmo #indexes into the column, wmo or float ID's
SMILE_floats_ND4 = SMILE_floats_D4.drop_duplicates() #gets rid of duplicates
SMILE_floats_ND4 #Tells you what these floats are

In [ ]:
len(SMILE_floats_ND4) #Tells you how many floats matched!

In [ ]:
#Region 2 - Argo floats that intersect EM-APEX SMILE experiment across all time & limited time
idx5 = ArgoIndexFetcher(src='erddap').region([-146.322, -145, 28.0, 30.6, '1999-01-01', '2023-01-01']).load()
idx6 = ArgoIndexFetcher(src='erddap').region([-146.322, -145, 28.0, 30.6, '2017-01-01', '2017-12-31']).load() #MOST INTERESTING

In [ ]:
SMILE_floats_D5 = idx5.index.wmo #indexes into the column, wmo or float ID's
SMILE_floats_ND5 = SMILE_floats_D5.drop_duplicates() #gets rid of duplicates
SMILE_floats_ND5 #Tells you what these floats are
len(SMILE_floats_ND5)

In [ ]:
SMILE_floats_D6 = idx6.index.wmo #indexes into the column, wmo or float ID's
SMILE_floats_ND6 = SMILE_floats_D6.drop_duplicates() #gets rid of duplicates
SMILE_floats_ND6 #Tells you what these floats are

In [ ]:
len(SMILE_floats_ND6) #Tells you how many floats matched!

In [ ]:
#Region 3 - Argo floats that intersect EM-APEX SMILE experiment across all time & limited time
34.0, 35.469333, -140.5, -139.39713
idx7 = ArgoIndexFetcher(src='erddap').region([-140.5, -139.39713, 34.0, 35.469333, '1999-01-01', '2023-01-01']).load()
idx8 = ArgoIndexFetcher(src='erddap').region([-140.5, -139.39713, 34.0, 35.469333, '2017-01-01', '2017-12-31']).load() #MOST INTERESTING

In [ ]:
SMILE_floats_D7 = idx7.index.wmo #indexes into the column, wmo or float ID's
SMILE_floats_ND7 = SMILE_floats_D7.drop_duplicates() #gets rid of duplicates
SMILE_floats_ND7 #Tells you what these floats are
len(SMILE_floats_ND7)

In [ ]:
SMILE_floats_D8 = idx8.index.wmo #indexes into the column, wmo or float ID's
SMILE_floats_ND8 = SMILE_floats_D8.drop_duplicates() #gets rid of duplicates
SMILE_floats_ND8 #Tells you what these floats are

In [ ]:
len(SMILE_floats_ND8)

#### Argo Float Matches!

After going through this indexing in the IndexFetcher, I was able to narrow down Argo float matches to Regions 1 and 2 in EM-APEX SMILE data that shared geographical space and time (2017):

Region #1:
5903608, 5904977, 4902947 (2017 isn't in geogrpahical bounds), 4902935 (2017 isn't in geographcial bounds), 5903603 (2017 isn't in geographical bounds)

Region #2:
5904128, 4902149

Region #3:
4900816, 4902251

And now we can move on to the graphs of each of these Argo floats!

### Option 3 - Downloading Argo Files onto your computer

You can also do this if you know what floats you want to focus on, but considering I had a lot of data to work with I did not use this method as much. The Argo Online School resources I mentioned in the .README file have more information about these methods and how Argo data files are organized if you want to try this method out. Good luck, and now on to the graphs!